Here, you develop the exact spatial derivative operator and its unit test. (Develop a quantitative assessment between what you've already found here for the Fourier derivative and the known derivative.)

In [ ]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML


sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST
from Modules import WaveformTools as WFT

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

Set parameters.

In [ ]:
nh = 32
refRatio = 2
c = 1.
CFL = 0.5
nt = 1
RK = 4
deriv = 'CD'
order = 0
wavenumber = 13


# Grid parameters.
nh_min = 64
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
CFL = np.sqrt(0.5)
nt = 50
deriv = 'CD'
RK = 4
order = 2

# Waveform parameters.
x_0 = 0.
x_1 = float(cfInter) / float(nh_min)
func = WFT.Gauss

# Physical parameters.
L = 1e-3
locs = []#x_1]
epsilons_r = [1]#, 11.68]
mus_r = [1]#, 0.99837]

# Pseudo-FFT parameters.
Hans = False

In [ ]:
omega = BT.Grid(nh)
finehalf = list(np.arange(int(nh / refRatio)) + int(nh / refRatio))
omega.AddPatch(refRatio, finehalf)

physics = BT.PhysProps(omega, epsilons_r, mus_r, locs, L)

waves = WT.MakeWaves(omega)
nullspace = OT.FindNullspace(omega, waves)
# cMat = physics.cMat
sigma, mu = WFT.GaussParams(x_0, x_1)
args = [sigma, mu]

In [ ]:
# For Gaussian:
waveform = WFT.InitCond(omega, physics, func, args)
waveDeriv = WFT.InitCond(omega, physics, func, args, deriv = True)

# For Gaussian wavepacket:
# waveform = WFT.WavePacket(omega, sigma, mu, wavenumber)
# waveDeriv = WFT.WavePacket(omega, sigma, mu, wavenumber, deriv = True)


In [ ]:
derivMat = OT.ExactSpatDerivOp(omega) # Works on fine grid to produce results on AMR grid!
spatOp = c * derivMat
# testDeriv = TT.ExactSpatDeriv(omega, t, waveform, -1, order)
# THIS IS A PROBLEM FOR NEW RK SETUP!!!

FCoefs = nullspace @ FFTT.FourierCoefs(waves @ nullspace, waveform)
FCoefsDeriv = nullspace @ FFTT.FourierCoefs(waves @ nullspace, waveDeriv)
FCoefsTestDeriv = spatOp @ FCoefs# FFTT.FourierCoefs(waves @ nullspace, testDeriv)
allCoefs = PT.Load(FCoefs, FCoefsDeriv, FCoefsTestDeriv)
labels = ['Initial Waveform', 'Exact Derivative', 'Fourier Derivative']

In [ ]:
PT.PlotMixedWave(omega, physics, waves, allCoefs, rescale = 6, yGrid = True, labels = labels)#, saveName = 'GaussTest', dpi = 300)


In [ ]:
what = (waves @ FCoefsTestDeriv) / (waves @ FCoefsDeriv)
print(FCoefsTestDeriv)
print(FCoefs)
print(what)
print('')